In [3]:
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
import numpy as np
wine = load_wine()
svc=SVC(kernel='linear')
cross_val_score(svc, wine.data, wine.target), np.mean(cross_val_score(svc, wine.data, wine.target))

(array([0.88888889, 0.94444444, 0.97222222, 1.        , 1.        ]),
 0.961111111111111)

In [5]:
cross_val_score(svc, wine.data, wine.target, cv=6), np.mean(cross_val_score(svc, wine.data, wine.target, cv=6))

(array([0.86666667, 0.9       , 0.93333333, 0.96666667, 1.        ,
        1.        ]),
 0.9444444444444445)

In [6]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [7]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=.2, train_size=.7,n_splits=10)
cross_val_score(svc, wine.data, wine.target, cv=shuffle_split)

array([0.91666667, 0.91666667, 0.91666667, 0.97222222, 1.        ,
       0.97222222, 0.94444444, 0.94444444, 0.94444444, 0.94444444])

In [8]:
from sklearn.model_selection import LeaveOneOut
cv = LeaveOneOut()
cross_val_score(svc, wine.data, wine.target, cv=cv)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1.])

## 简单网格搜索 

In [9]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, random_state=38)
best_score = 0
for alpha in [0.01,0.1,1,10]:
    for max_iter in [100, 1000, 5000, 10000]:
        lasso = Lasso(alpha=alpha, max_iter=max_iter)
        lasso.fit(x_train, y_train)
        score = lasso.score(x_test, y_test)
        if score > best_score:
            best_score = score
            best_parameters = (alpha, max_iter)
best_score, best_parameters

(0.8885499702025689, (0.01, 100))

## 与交叉验证结合的网格搜索

In [10]:
best_score = 0
for alpha in [0.01,0.1,1,10]:
    for max_iter in [100, 1000, 5000, 10000]:
        lasso = Lasso(alpha=alpha, max_iter=max_iter)
        scores = cross_val_score(lasso, x_train, y_train, cv=6)
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = (alpha, max_iter)
best_score, best_parameters

(0.8533004713012285, (0.01, 100))

In [11]:
lasso = Lasso(alpha=0.01, max_iter=100).fit(x_train, y_train)
lasso.score(x_test, y_test)

0.8885499702025689

In [13]:
from sklearn.model_selection import GridSearchCV
params = {'alpha':[0.01,0.1,1,10],
         'max_iter':[100,1000,5000,10000]}
grid_search = GridSearchCV(lasso, params, cv=6)
grid_search.fit(x_train, y_train)
grid_search.score(x_test, y_test),grid_search.best_params_, grid_search.best_score_

(0.8885499702025689, {'alpha': 0.01, 'max_iter': 100}, 0.8533004713012285)

使用 `predict_proba`方法预测模型的准确率。（一个数据点属于一个类别的概率。）,并不是每个分类函数都有该属性，此时需要使用`decision function`函数

In [14]:
svc = SVC().fit(x_train, y_train)
dec_fun = svc.decision_function(x_test)
dec_fun[:5]

array([[-0.24437122,  2.20443906,  1.17907053],
       [ 2.23188277, -0.23764364,  1.05504757],
       [ 2.23983534, -0.24143095,  1.01942999],
       [ 2.2245817 , -0.23413999,  1.07600311],
       [-0.2456163 ,  2.20772349,  1.17803295]])